<a href="https://colab.research.google.com/github/hannanabdul55/neural-nets-research-playground/blob/master/White_house_briefings_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a Whitehouse press briefings Model

This is based off a notebook created by [Max Woolf](http://minimaxir.com) originally (so most of the credits go there!) and updated by [Abdul Hannan](https://abdulhannan.in).  

The license is [here](#scrollTo=wmTXWNUygS5E).

*Last updated: May 3, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using Max Woolf's `gpt-2-simple` library!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read this [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

One of the main advantages of Google Colaboratory is that you can GPU access for free! Yes, for FREE! Hence, this is the perfect places to train your model from scratch.

Run this command to see if there is a GPU connected to this instance.  

In [0]:
print('GPU connected?\n The answer is: ', tf.test.is_gpu_available())

NameError: ignored

If the answer is `False`, then go to Runtime -> Change Runtime Type -> Under 'Hardware Accelerator', choose 'GPU'. Then re-run this notebook from the beginning. 

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by selecting the ceel from the dropdown form in the next cell.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
#@title Choose how big a model you want to train.
#@markdown Bigger model takes more GPU space and time to train! 
model_name = "355M" #@param ["124M", "355M", "774M", "1558M"]

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 361Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 73.1Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 223Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:04, 110Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 254Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 127Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 149Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
# import some packages which will be used
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

# define some methods which will be used to extract data from the HTML websites

def get_html_text(url_p):
    print('Reading URL: %s'%(url_p))
    html_t = urllib.request.urlopen(url_p).read()
    soup_t = BeautifulSoup(html_t)
    # kill all script and style elements
    for script in soup_t(["script", "style", 'aside', 'head']):
        script.extract()    # rip it out
    return soup_t.body.main

def get_soup(url_p):
    html_t = urllib.request.urlopen(url_p).read()
    soup_t = BeautifulSoup(html_t)
    # kill all script and style elements
    for script in soup_t(["script", "style", 'aside', 'head']):
        script.extract()    # rip it out
    return soup_t


In [0]:
url_list = 'https://www.whitehouse.gov/search/?s=Coronavirus+Task+Force&wpsolr_fq%5B0%5D=issue_str%3AHealthcare'
base_url = 'https://www.whitehouse.gov'
soup_l = get_soup(url_list)

In [0]:
urls = []
c_url = 'https://www.whitehouse.gov/search/?s=Coronavirus+Task+Force&wpsolr_fq%5B0%5D=issue_str%3AHealthcare'
page_num = 0
while True:
    page_num+=1
    soup_l = get_soup(c_url)
    for s in soup_l.find_all('article'):
        link = s.find("h2", {'class': 'briefing-statement__title'})
        if link is not None:
            urls.append(link.a['href'])
            # print(link.a['href'])
    next = soup_l.find("a", {'class':'pagination__next'})
    # print(next)
    if next is None:
        break;
    c_url = base_url + next.get('href')
print('Number of pages parsed: %d'%(page_num))
# print(urls[1])
samples = list(map(lambda x: get_html_text(x).get_text().strip(), urls))

Number of pages parsed: 9
Reading URL: https://www.whitehouse.gov/briefings-statements/readout-white-house-coronavrius-task-force-meeting/
Reading URL: https://www.whitehouse.gov/briefings-statements/press-briefing-members-presidents-coronavirus-task-force/
Reading URL: https://www.whitehouse.gov/briefings-statements/remarks-president-trump-members-coronavirus-task-force-press-briefing-3/
Reading URL: https://www.whitehouse.gov/briefings-statements/remarks-president-trump-members-coronavirus-task-force-press-briefing-2/
Reading URL: https://www.whitehouse.gov/briefings-statements/remarks-president-trump-members-coronavirus-task-force-press-briefing/
Reading URL: https://www.whitehouse.gov/briefings-statements/remarks-president-trump-vice-president-pence-members-coronavirus-task-force-press-briefing-10/
Reading URL: https://www.whitehouse.gov/briefings-statements/vice-president-pence-secretary-azar-add-key-administration-officials-coronavirus-task-force-2/
Reading URL: https://www.white

In [0]:
#@markdown ### Enter a name for the file that would be used to save the extracted text to Google Drive:
op_file = 'input2.txt' #@param {type: "string"}
def save_text_gdrive(filename, samples):
    with open('/content/drive/My Drive/%s' % (filename), 'w') as f:
        for inp in samples:
            f.write(inp + '\n\n')

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.  

Now that you have saved the file to Google Drive, you can now use this file whenever you want for any other tasks too!  

In [0]:
#@markdown Enter the name of the file in google Drive for the text on which you want to train.
file_name = 'input2.txt' #@param {type: "string"}
gpt2.copy_file_from_gdrive(file_name)

FileNotFoundError: ignored

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
#@markdown Give a name for your run. This will be used as a name to store all the intermediate text, mdoels and checkpoints.
run_name = 'run1' #@param {type:"string"}

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from='fresh',
              run_name=run_name,
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:04<00:00,  4.90s/it]


dataset has 1113153 tokens
Training...
[10 | 30.61] loss=2.53 avg=2.53
[20 | 52.48] loss=2.53 avg=2.53
[30 | 74.37] loss=2.47 avg=2.51
[40 | 96.29] loss=2.41 avg=2.48
[50 | 118.17] loss=2.24 avg=2.43
[60 | 140.12] loss=2.43 avg=2.43
[70 | 162.11] loss=2.51 avg=2.44
[80 | 184.12] loss=2.26 avg=2.42
[90 | 206.18] loss=2.16 avg=2.39
[100 | 228.24] loss=2.09 avg=2.36
[110 | 250.31] loss=2.13 avg=2.34
[120 | 272.36] loss=2.26 avg=2.33
[130 | 294.36] loss=2.24 avg=2.32
[140 | 316.42] loss=2.31 avg=2.32
[150 | 338.44] loss=2.01 avg=2.30
[160 | 360.47] loss=2.63 avg=2.32
[170 | 382.51] loss=2.21 avg=2.31
[180 | 404.54] loss=2.15 avg=2.30
[190 | 426.60] loss=2.68 avg=2.33
[200 | 448.64] loss=2.50 avg=2.34
======== SAMPLE 1 ========
 it was so much fun.  It wasn’t as glamorous.  But it was a great experience.  A lot of people came out for it — like me, myself — and they wanted to be in the hotel like everybody else.  They wanted to be in.  The people wanted to have it.  A lot of people wanted to

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
#@markdown Add the name of the run which you stored on to your Google Drive after running the above set of cells.
#@markdown **PS** if you have run the training part of the section above, you do not have to run this cell!  
run_name = run_name #@param {type: "string"}

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name=run_name)

The United Nations has been working very hard.  They’ve been working for a long time, as you know.  We’re now at a point where we have to call off the assembly, which unfortunately took place this morning.  We’ll be doing it in the future.
The United Nations has been doing excellent work.  They’ve been talking about everything.
In the House, they’ve had a lot to say.  They’ve had a lot of discussion.  There’s a lot of people talking to each other.  There’s a lot of people that are really getting together, and they’re talking about things that are very important.
We have a great relationship with China.  They had a very short period of time ago.  We’ve had a very good relationship.  We’re making a lot of progress there.  And they’ve been — in exchange for allowing us to do this, we’re going to be bringing our ships back to Guam and our air bridge back into the United States.
I just spoke to President Xi.  I had a very good talk with the President of China.  It was a long talk.  And I’m 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
#@markdown Here, change the prefic to whatever you like, I have kept this so that it generates something like the press house briefings! It is a good experiment to try removing it completely and seeing what you get 😊
gpt2.generate(sess,
              length=1023,
              temperature=0.7,
              prefix="Q  Mr. President,",
              nsamples=5
            #   batch_size=2
              )

Q  Mr. President, 24 states — 24 states — in addition to New York and 34 before — reported 100 or more test positive for coronavirus.  Around 1,200 of those — 34 — are already home.  Can you reassure Americans that those are clean settings and that the coronavirus is going to be contained?
THE PRESIDENT:  So, when Governor Cuomo was saying in the last time we were here — and I can assure you, Chris, governors across the country, including governor Chin, saying, �We’ve had 100 or more tests.  We’ve had no cases.  This is like — you know, we hope it’s going to go away.  But it’s going to come back because we had a great testing system.  We had more than enough testing to go — as many as we could.  And then we moved over to more — we moved over to —
Q    And when did you make the decision to go on a nationwide exclusion?
THE PRESIDENT:  — statewide exclusion?
Q    No, no, not nationwide exclusion.  I mean, the states didn’t know the virus existed.  And the CDC — and I’m sure you know, Bre

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.